In [ ]:
from pathlib import Path

In [ ]:
# You can ignore about the `pydantic` deprecation warning (coming from `tiled`)
import pamila as pml
from pamila import Q_

In [ ]:
# Activate the simulator mode (i.e., neither LIVE nor DT [Digital Twin])
pml.go_offline()

In [ ]:
facility_name = pml.machine.get_facility_name()
FACILITY_CONFIG_FOLDER = Path("demo_generated") / facility_name

In [ ]:
machine_name = "SR"
SR = pml.load_machine(machine_name, dirpath=FACILITY_CONFIG_FOLDER)
SR

In [ ]:
mlvs = SR.get_all_mlvs()
mlvs

In [ ]:
for mlv in mlvs.values():
    mlv.wait_for_connection()

In [ ]:
# "C30_C1" refers to the first orbit corrector in Cell 30 at NSLS-II.
mlvs["C30_C1_x_I_RB"].get(), mlvs["C30_C1_x_I_SP"].get()

In [ ]:
mlvs["C30_C1_x_angle_SP"].get(), mlvs["C30_C1_x_angle_RB"].get()

Note that the units shown above are "radian", not "mrad", which was specified in "elements.yaml" for the "x_angle" repr. for the MLV "C30_C1_x_angle_SP".

The units of the angle in `pyAT` is "radian". The PAMILA signal attaches "radian" to the float value obtained from the simulator.

Normally, the repr. conversion functions that involve the "x_angle" repr. will convert the object to "mrad" before passing as an input argument, or, if the repr. is an output, then the function attaches "mrad" to the output float value.

In this particular case, however, there was no repr. conversion (i.e., identity conversion from "x_angle" to "x_angle"). Since no conversion process was invoked, the originally attached unit of "radian" was left as is.

In [ ]:
mlv = mlvs["C30_C1_x_I_SP"]

pdev = mlv.get_device()

pdev._machine_name, pdev._mode

In [ ]:
# Switch to DT mode.
# Notice that MLV now points to the pamila device (pdev) for DT, not SIM.
pml.set_online_mode(pml.MachineMode.DIGITAL_TWIN)
pml.go_online()
pdev = mlv.get_device()

pdev._machine_name, pdev._mode

In [ ]:
# Go back to the simulator mode
pml.go_offline()

In [ ]:
mlv.name

In [ ]:
mlv.get()

In [ ]:
mlv.read()

In [ ]:
# Should result in a TypeError
try:
    mlv.put(0.1)
except Exception as e:
    assert isinstance(
        e, TypeError
    ), f"Expected TypeError, but got {type(e).__name__}"
    assert str(e) == "Wrong type: <class 'float'>"
    print("Failed as expected!")
    print("Must pass a pint.Q_ object, instead of a float object")
except:
    raise

In [ ]:
# You must pass a pint Quantity object `Q_`.
mlv.put(Q_("0.1 A"))

In [ ]:
# Confirm that the value for the MLV has been changed
mlv.get()

In [ ]:
# Note that the setpoint value in "x_angle" repr. is also no longer zero radian.
mlvs["C30_C1_x_angle_SP"].get()

In [ ]:
# Check out other MLVs
dcct_mlv = SR.get_mlv("Beam_Current_I_RB")
nux_mlv = SR.get_mlv("BxB_Tune_nux_RB")
nuy_mlv = SR.get_mlv("BxB_Tune_nuy_RB")

In [ ]:
dcct_mlv.get()

In [ ]:
nux_mlv.get()

In [ ]:
nuy_mlv.get()